# Analyze combined states (different models)

In [1]:

""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import seaborn as sns
from collections import defaultdict
import pandas as pd

from one.api import ONE
from jax import vmap
from pprint import pprint
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Models/Sub-trial//2_fit_models/'
#functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability//Models/Sub-trial//2_fit_models/'
os.chdir(functions_path)
from preprocessing_functions import idxs_from_files, prepro_design_matrix, concatenate_sessions, fix_discontinuities
from fitting_functions import cross_validate_armodel, compute_inputs
from plotting_functions import plot_transition_mat, plot_states_aligned, align_bin_design_matrix, states_per_trial_phase, plot_states_aligned_trial, traces_over_sates, traces_over_few_sates

one = ONE()

Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/tmpay2uic0_/cache.zip Bytes: 127561276


  7%|▋         | 8.0/121.65191268920898 [08:17<1:57:43, 62.15s/it]


KeyboardInterrupt: 

## Parameters

In [ ]:
bin_size = 0.1

# Plotting params
multiplier = 1/bin_size

event_type_list = ['goCueTrigger_times']  # , 'feedback_times', 'firstMovement_times'
event_type_name = ['Go cue']  # , 'Feedback time', 'First movement onset'


## Data path

In [ ]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v4_5Jul2024/' + str(bin_size) + '/'
results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/'

os.chdir(data_path)
design_matrices = os.listdir(data_path)

var_sets = [['avg_wheel_vel'], ['nose_X', 'nose_Y'], ['Lick count'], 
                    ['left_X', 'left_Y', 'right_X', 'right_Y'], ['whisker_me']]
var_sets = [['avg_wheel_vel'], ['nose_X', 'nose_Y'], 
                    ['left_X', 'left_Y', 'right_X', 'right_Y'], ['whisker_me']]
    
# Load frame rate
fr_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/frame_rate/'
os.chdir(fr_path)
frame_rate = pickle.load(open(fr_path + "frame_rate", "rb"))


AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/home/ines/miniconda3/envs/glmhmm/lib/python3.7/site-packages/pandas/_libs/internals.cpython-37m-x86_64-linux-gnu.so'>

## Get mice list

In [ ]:
idxs, mouse_names = idxs_from_files(one, design_matrices, frame_rate, data_path, bin_size)

local md5 mismatch on dataset: churchlandlab/Subjects/CSHL049/2020-01-11/001/alf/_ibl_trials.stimOff_times.npy
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/churchlandlab/Subjects/CSHL049/2020-01-11/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 4.60k/4.60k [00:00<00:00, 9.93kB/s]
local md5 mismatch on dataset: cortexlab/Subjects/KS014/2019-12-03/001/alf/_ibl_trials.goCueTrigger_times.npy
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/cortexlab/Subjects/KS014/2019-12-03/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 4.38k/4.38k [00:00<00:00, 9.45kB/s]


## Parameters

In [ ]:
num_iters = 100
num_train_batches = 5
method = 'kmeans'
threshold = 0.05

num_lags = 11  # First order ARHMM
num_states = 2
kappa = 1000

# Plotting params
bin_size = 0.1

# Values for grid search (should get this from results)
last_lag = 20
lag_step = 2
start_lag = 1
Lags = list(range(start_lag, last_lag, lag_step))
kappas = [0, 1, 5, 10, 100, 500, 1000, 2000, 5000, 7000, 10000]

## Get combined states from mice

In [ ]:
# Get data for all mice for the model of interest
os.chdir(data_path)
matrix_all, matrix_all_unnorm, session_all = prepro_design_matrix(one, idxs, mouse_names, 
                                                                    bin_size, var_names, data_path, first_90=True)
collapsed_matrices, collapsed_unnorm, collapsed_trials = concatenate_sessions(mouse_names, matrix_all, 
                                                                                matrix_all_unnorm, session_all)

# Loop through animals
for m, mat in enumerate(idxs[0:1]):
    if len(mat) > 35: 
        mat = '46794e05-3f6a-4d35-afb3-9165091a5a74_CSHL045'
        
        mouse_name = mat[37:]
        session = mat[0:36]
        
        most_likely_states = pickle.load(open("most_likely_states" + var_names[0] + '_' + mouse_name, "rb"))
        
        # Get design_matrix
        design_matrix = collapsed_matrices[mouse_name]
        if len(np.shape(design_matrix)) > 2:
            design_matrix = design_matrix[0]

        